# БАЗА

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv("/kaggle/input/dataset/ner_data_train.csv")

In [3]:
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

# **Обучение:**

In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_lg-3.1.0/ru_core_news_lg-3.1.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.9/513.9 MB 2.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 73.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 17.0 MB/s eta 0:00:00
  Created wheel for ru-core-news-lg: filename=ru_core_news_lg-3.1.0-py3-none-any.whl size=514191300 sha256=c112ed2fc05145a394448965521695302b091e624c1f2ef0b1e5a04940c6c2ff
  Stored in directory: /root/.cache/pip/wheels/b8/bb/c1/eef901e678c3e3356bd67ee050423b883e6227666074a554c4
Successfully built ru-core-news-lg
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninst

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
data = pd.read_csv("/kaggle/input/dataset/ner_data_train.csv")

In [13]:
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [14]:
import spacy
from spacy.training.example import Example
nlp = spacy.load("ru_core_news_lg")

In [15]:
ner = nlp.get_pipe("ner")

In [16]:
ner.add_label("видеоигра")
ner.add_label("серия")
ner.add_label("персона")
ner.add_label("вид спорта")
ner.add_label("название проекта")
ner.add_label("сезон")
ner.add_label("организация")
ner.add_label("команда")
ner.add_label("бренд")
ner.add_label("лига")
ner.add_label("локация")
ner.add_label("модель")
ner.add_label("Дата")
ner.add_label("неизвестно")

1

In [20]:
x = df['video_info']
y = df['entities']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
train_data = []
for text, entities in zip(x_train, y_train):
    entity_info = [(entity['offset'], entity['offset'] + entity['length'], entity['label']) for entity in entities]

    example = Example.from_dict(nlp.make_doc(text), {'entities': entity_info})
    train_data.append(example)


/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "<НАЗВАНИЕ:> Бэтмен против Супермена: На заре справ..." with entities "[(12, 18, 'персона'), (26, 35, 'персона'), (37, 59...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "<НАЗВАНИЕ:> НОЧНОЙ АКВАПАРК единственный в России!..." with entities "[(1, 9, 'не найдено'), (12, 27, 'организация'), (7...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be 

In [22]:
for example in train_data:
    nlp.update([example], drop=0.5) 

In [23]:
nlp.to_disk("/kaggle/working/nlp")

# Распознавание

In [29]:
import spacy
from spacy.training.example import Example
nlp = spacy.load("/kaggle/working/nlp")

In [32]:
predicted_labels = []
for text in x_test:
    doc = nlp(text)
    labels = [ent.label_ if ent.label_ else "не известно" for ent in doc.ents]
    predicted_labels.append(labels)

In [34]:
true_labels = []
for element in y_test:
    subdata = []
    for subelement in element:
        subdata.append(subelement['label'])
    true_labels.append(subdata)

In [37]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

In [40]:
mlb = MultiLabelBinarizer()
true_binary = mlb.fit_transform(true_labels)
predicted_binary = mlb.transform(predicted_labels)

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [41]:
f1 = f1_score(true_binary, predicted_binary, average="weighted")

print(f"F1-score: {f1:.2f}")

F1-score: 0.54


In [24]:
!zip -r file.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/nlp/ (stored 0%)
  adding: kaggle/working/nlp/tokenizer (deflated 83%)
  adding: kaggle/working/nlp/parser/ (stored 0%)
  adding: kaggle/working/nlp/parser/cfg (deflated 32%)
  adding: kaggle/working/nlp/parser/moves (deflated 49%)
  adding: kaggle/working/nlp/parser/model (deflated 7%)
  adding: kaggle/working/nlp/tok2vec/ (stored 0%)
  adding: kaggle/working/nlp/tok2vec/cfg (stored 0%)
  adding: kaggle/working/nlp/tok2vec/model (deflated 7%)
  adding: kaggle/working/nlp/senter/ (stored 0%)
  adding: kaggle/working/nlp/senter/cfg (stored 0%)
  adding: kaggle/working/nlp/senter/model (deflated 9%)
  adding: kaggle/working/nlp/ner/ (stored 0%)
  adding: kaggle/working/nlp/ner/cfg (deflated 33%)
  adding: kaggle/working/nlp/ner/moves (deflated 90%)
  adding: kaggle/working/nlp/ner/model (deflated 7%)
  adding: kaggle/working/nlp/attribute_ruler/ (stored 0%)
  adding: kaggle/working/nlp/attribute_ruler/patterns (deflated 76%)
